# Analyzing Discord Messages 

Import df and filter out wrong Processes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("dist\input_log.csv")
df = df[df['Process'] == 'Discord.exe']
df.head()

Check all unique values -> See which ones we want to sort out

In [ ]:
df["Button"].unique()

## Plotting amount of presses per Button

Creating a List of buttons we do not care about, basically all non letters/numbers that don't show up in actual messages + symbols

In [ ]:
# Non letter character list
notWanted = ["alt", "LC", "RC", "CTRL", "esc", "Tab", "SH", "Enter", "<-", " ", '\'', '.','?','!']
# Create lettercount_df = Filter out the notWanted list
lettercount_df = df[~df["Button"].isin(notWanted)]
lettercount_df['Button'] = lettercount_df['Button'].str.lower()

# Get value counts of the cleaned-up 'Button' column and sort by index
lettercount_df = lettercount_df['Button'].value_counts().sort_index().sort_values(ascending=False)

# Create a bar plot
plt.figure(figsize=(10, 5))
plt.bar(lettercount_df.index, lettercount_df.values, color="skyblue")

# Add labels and title
plt.xlabel("Letter")
plt.ylabel("Frequency")
plt.title("Letter Usage Frequency in Process")
plt.xticks(rotation=0)  # Keep letters horizontal

# Show the plot
plt.show()

## Read Discord messages

Putting together the messages 1 by 1 by using the enter presses as delimiter = message send. Saving them in a df for further use

In [ ]:
cleaned_buttons = []
notWantedReconstruct = ["alt", "LC", "RC", "CTRL", "esc", "Tab", "SH", '\'', '.','!']
dfReconstruct = df[~df["Button"].isin(notWantedReconstruct)]
df_messages = pd.DataFrame(columns=["Message"])

# Simulate the backspace logic
for button in dfReconstruct['Button']:
    if button == "<-":
        if cleaned_buttons:
            cleaned_buttons.pop()  # Remove the last valid button
    else:
        cleaned_buttons.append(button)

# Now reconstruct messages using Enter as delimiter
messages = []
current_message = []

for button in cleaned_buttons:
    if button.lower() == "enter":
        if current_message:
            message_str = ''.join(current_message)  # Turn list into single string
            messages.append(message_str)
            current_message = []
    else:
        current_message.append(button)

# Add last message if Enter wasn't pressed at the end
if current_message:
    messages.append(''.join(current_message))

# Convert to DataFrame where each row is one full message string
df_messages = pd.DataFrame(messages, columns=["Message"])
df_messages

# Print or save
